## Load Disaster-related Statistics (BBS, 2015)

In [162]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer
import fhv
from tabula import read_pdf
# zila = gpd.read_file('./data/admin_boundary/bgd_admbnda_adm2_bbs_20180410.shp')
# zila = zila[['ADM1_EN','ADM1_PCODE','ADM2_EN','ADM2_PCODE']]
# zila[['ADM1_PCODE','ADM2_PCODE']] = zila[['ADM1_PCODE','ADM2_PCODE']].astype(int)
# mymensingh = (zila['ADM1_PCODE'] == 45)
# zila.loc[mymensingh, 'ADM2_PCODE'] = zila.loc[mymensingh, 'ADM2_PCODE'] % 100 + 3000
# zila = zila.sort_values(by='ADM2_PCODE').reset_index(drop=True)
# zila.to_excel('./data/zila_list.xlsx')

### List of extracted tables from the report
- Table 4: Distribution of household by main source of income and received remittance by division and district, 2014.
- Table 5: Distribution of main source oflighting and cooking fuel by division and district, 2014.
- Table 18: Distribution of annual household income from agricultural products by division and district, 2014.
- Table 20: Distribution of annual household income from non-agricultural sector by division and district, 2014.
- Table 22: Distribution of annual household income from other source by division and district, 2014.
- Table 23: Distribution of Disaster affected times of household by division, 2009-'14.
- Table 24: Distribution of affected households by disaster categories by division, 2009-'14.
- Table 25: Distribution of affected household and disaster categories by division and district, 2009-'14.
- Table 26: Distribution of household number of non working days due to last natural disaster by disaster categories and division, 2009-'14.
- Table 27: Distribution of Affected Household got early warning by disaster categories and division, 2009-'14.
- Table 28: Distribution of household got early warning by type of media, disaster categories and division, 2009-'14.
- Table 29: Distribution of affected area and loss of major crops by type of disaster categories and division, 2009-'14    
- Table 30: Distribution of affected area and value of loss and damage of minor crops by type of disaster categories and division, 2009-'14.
- Table 31: Distribution of affected area and loss of major crops by division and district, 2009-'14.
- Table 32: Distribution of affected area and loss of minor crops by division and district, 2009-'14.
- Table 35: Distribution of area and damage value of land by disaster categories and division, 2009-'14.
- Table 36: Distribution of area and damage value of land by division and district, 2009-'14.
- Table 39: Distribution of population suffering from sickness and injury by sex, disaster categories and division, 2009-'14.
- Table 40: Distribution of population suffering from sickness and injury by sex, age group and division, 2009-'14.
- Table 41: Distribution of population suffering from sickness and injury by sex, division and district, 2009-'14.  
- Table 42: Distribution of number of total children and sick children by division and district, 2009-'14.
- Table 48: Distribution of Children did not attend to School Due to Natural Disaster by Division and District, 2009-'14.
- Table 51: Distribution of disaster preparedness of household by disaster category and division, 2009-'14.
- Table 52: Distribution of disaster preparedness of household by division and district, 2009-'14.
- Table 53: Distribution of households having disaster precaution measures according to prior-disaster experience by disaster and division, 2009-'14.
- Table 54: Distribution of household preparedness during disaster period untill normal situation by disaster and division, 2009-'14.
- Table 55: Distribution of household preparedness during disaster period untill normal situation by division and district, 2009-'14.
- Table 56: Distribution of household taken action (precaution) during disaster period until normal situation by disaster and division, 2009-'14.
- Table 57: Distribution of population suffering from disease due to disaster by division and district, 2014.
- Table 58: Distribution of population suffering from disease due to natural disaster by sex, age group, division and district, 2014.
- Table 59: Distribution of Population Suffering from Disease Due to natural disaster by Type of Disease, Division and District, 2014.    
- Table 60: Distribution of household members suffering from disease before disaster by division and district, 2009-'14.
- Table 61: Distribution of household members suffering from disease during disaster period by division and district, 2009-'14
- Table 62: Distribution of household members suffering from disease post disaster period by division and district, 2009-'14.
- Table 63: Distribution of main probable cause of suffering from disease due to disaster by division and district, 2014.    
- Table 64: Distribution of source of household drinking water during disaster period by division and district, 2009-'14.
- Table 65: Distribution of other use of water (cooking, sewerage, cleanliness etc.) before disaster period by division and district, 2009-'14.
- Table 66: Distribution of other use water (cooking, sewerage, cleanliness etc.) during disaster period by division and district, 2009-'14.
- Table 67: Distribution of disease status due to insufficient drinking and other use of water supply during/after disaster period by division and district, 2009-'14.    
- Table 68: Distribution of cause of main disease due to insufficient drinking and other use of water supply during/after disaster period by division and district, 2009-'14.
- Table 71: Distribution of respondent's knowledge and perception about main impact of climate change by division and district, 2014.
- Table 72: Distribution of respondent's knowledge and perception about disaster by division and district, 2014
- Table 73: Distribution of Respondent's knowledge and perception about disaster management by division and district, 2014.
- Table 74: Distribution of household received finantial/rehabiltation support from government/non-government agency during/post disaster period by division and district, 2009-'14
- Table 75: Distribution of household received financial/rehabilitation support from different organization/ office during/post disaster period by division and district, 2009-'14.
- Table 76: Distribution of households received loan from post disaster period by division and district, 2009-'14.
- Table A1: Standard error calculate of total income and total damage and loss by divisiond and district.

In [294]:
DistrictName = ['Barisal','Chittagong','Dhaka','Khulna','Rahshahi','Rangpur','Sylhet']
DisasterType = ['Drought','Flood','Water logging','Cyclone',
                'Tornado','Storm/Tidal Surge','Thunderstorm','River/Coastal Erosion',
                'Landslides','Salinity','Hailstorm','Others']
Mulcol = pd.MultiIndex.from_product([DistrictName, DisasterType], names=['District','Disaster'])
df = pd.read_excel('./data/Disaster-related Statistics 2015.xlsx', sheet_name='Zila')
rind = np.array([0,1,8,20,38,49,58,67])
dist_new_name = df['ADM2_EN']

def LoadDisasterStat2015(sheet_name):
    
    # For the last table (Table_A1)
    if sheet_name == 'Table_A1':
        df = pd.read_excel('./data/Disaster-related Statistics 2015.xlsx', 
                   sheet_name='Table_A1', 
                   skiprows=1,
                   header=[0])
        df = df.set_index('Division/District')
        df.index.name = 'District'
        df.index = dist_new_name
        return df

    # For other tables
    df = pd.read_excel('./data/Disaster-related Statistics 2015.xlsx', 
                       sheet_name=sheet_name, 
                       skiprows=1,
                       header=[0,1])    
    if df.columns[0][0] == 'Division/District':
        # Length of single columns
        ind = len([name for name in df.columns.get_level_values(1).astype(str) if 'Unname' in name])
        sub1 = df[df.columns[:ind]]
        sub2 = df[df.columns[ind:]]
        sub1.columns = pd.MultiIndex.from_tuples([(c[0], '') for c in df[df.columns[:ind]] ])
        df = pd.concat([sub1,sub2], axis=1).set_index('Division/District')
        assert df.isna().sum().sum() == 0

        # Reshape dataframe
        df.index.name = 'District'
        assert df.shape[0] == 72
        df = df.drop(df.iloc[rind].index, axis = 0)
        df.index = dist_new_name
        # df = df.drop(['Total Household'], axis=1)

    elif df.columns[0][0] == 'Type of Disaster':
        # Length of single columns
        ind = len([name for name in df.columns.get_level_values(1).astype(str) if 'Unname' in name])
        sub1 = df[df.columns[:ind]]
        sub2 = df[df.columns[ind:]]
        sub1.columns = pd.MultiIndex.from_tuples([(c[0], '') for c in df[df.columns[:ind]] ])
        df = pd.concat([sub1,sub2], axis=1).set_index('Type of Disaster')

        # Reshape dataframe
        df = df[~(df.isna().sum(1) == df.shape[1])]  # Select empty rows
    #     df = df.drop(['Total Household'], axis=1)
        df = df[df.index != 'Total']
        df = df.iloc[12:]    # Remove Bangladesh total
        assert df.shape[0] == 84
        df = pd.DataFrame(data=df.values,index=Mulcol,columns=df.columns.get_level_values(1))
        assert df.isna().sum().sum() == 0
        
    return df

In [295]:
df = LoadDisasterStat2015('Table_A1')
df

,Total income,Total damage and loss
ADM2_EN,,
Barguna,122967.1,32280.14
Barisal,149358.1,69810.39
Bhola,169081.9,56194.17
Jhalokati,152055.1,29269.47
Patuakhali,138763.8,40681.59
...,...,...
Thakurgaon,143977.2,28298.21
Habiganj,126606.8,31651.55
Maulvibazar,123001.3,20428.92


In [ ]:
### Type of House and Tenancy

In [284]:
df = pd.read_excel('./data/union/Type of House and Tenancy.xls',
                   skiprows=11,header=0,index_col=0,skipfooter=8)

df

,Union,Type of House,Tenancy of House,Unnamed: 4,Unnamed: 5,Unnamed: 6
NaN,Ward no-01,NaN,Own,Rented,Rent-free,Total
NaN,NaN,Pucca,14,3,2,19
NaN,NaN,Semi-pucca,10,1,-,11
NaN,NaN,Kutcha,191,3,2,196
NaN,NaN,Total,215,7,4,226
...,...,...,...,...,...,...
NaN,NaN,Pucca,1859653,1780168,109731,3749552
NaN,NaN,Semi-pucca,4371772,1827686,154614,6354072
NaN,NaN,Kutcha,19556239,890946,692832,21140017
NaN,NaN,Jhupri,703423,100371,126185,929979
